# keras example - telco customer churn dataset

This lesson uses the [telco customer churn dataset](course_datasets.md#telco-customer-churn).

A similar tutorial, based on the same dataset, is [here](https://cocalc.com/github/codebasics/deep-learning-keras-tf-tutorial/blob/master/11_chrun_prediction/churn.ipynb).

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from sklearn.metrics import accuracy_score
from tensorflow.keras.models import Sequential, load_model
from tensorflow.keras.layers import Dense

In [ ]:
df = pd.read_csv('https://raw.githubusercontent.com/MarkWilcock/Python-ML-Course/refs/heads/main/data/telco_customer_churn.csv') # if running on Colab
#df = pd.read_csv('data/telco_customer_churn.csv') # if running locally
df.head(2)

In [ ]:
X = df.drop(columns=['Churn', 'Customer ID'])
y = (df['Churn'] == 'Yes').astype(int)
(X.shape, y.shape)

In [ ]:
# diagnostics
X.head(2)

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2)

# Apply get_dummies after splitting to avoid data leakage
X_train = pd.get_dummies(X_train)
X_test = pd.get_dummies(X_test).reindex(columns=X_train.columns, fill_value=0)

(X_train.shape, y_train.shape), (X_test.shape, y_test.shape)

In [ ]:
# diagnostics
X_train.shape[1]

In [ ]:
model = Sequential()
model.add(Dense(units=32, input_dim=X_train.shape[1], activation='relu'))
model.add(Dense(units=64, activation='relu'))
model.add(Dense(units=1, activation='sigmoid'))

In [ ]:
model.compile(loss='binary_crossentropy', optimizer='sgd', metrics=['accuracy'])

In [ ]:
model.fit(X_train, y_train, epochs=200, batch_size=32, verbose=0)

In [ ]:
y_hat = (model.predict(X_test) > 0.5).astype(int).flatten()
y_hat[:5]

In [ ]:
accuracy_score(y_test, y_hat)

In [ ]:
keras_model_file_path = './outputs/model.keras'
model.save(keras_model_file_path)
del model
model = load_model(keras_model_file_path)
model